In [2]:
import pandas as pd
import numpy as np

# Categorical Data

## Background and Motivation

Kategorični podatki omogočajo boljšo performanco in uporabo memorija

In [3]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [5]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [6]:
values.value_counts

<bound method IndexOpsMixin.value_counts of 0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object>

In [8]:
values = pd.Series([0, 1, 0, 0] * 2)

In [9]:
dim = pd.Series(['apple', 'orange'])

In [10]:
dim

0     apple
1    orange
dtype: object

> [pandas.Series.take](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.Series.take.html)

## Categorical Type in pandas

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N = len(fruits)

In [13]:
df = pd.DataFrame({'fruit': fruits,
    'basket_id': np.arange(N),
    'count': np.random.randint(3, 15, size=N),
    'weight': np.random.uniform(0, 4, size=N)},
    columns=['basket_id', 'fruit', 'count', 'weight'])

In [14]:
df

,basket_id,fruit,count,weight
0,0,apple,3,2.606553
1,1,orange,6,1.186622
2,2,apple,5,2.503871
3,3,apple,5,3.888990
4,4,apple,6,3.601628
5,5,orange,12,1.177738
6,6,apple,8,1.603607
7,7,apple,10,2.287510


In [17]:
df["fruit"] = df["fruit"].astype("category")
df.dtypes

basket_id       int64
fruit        category
count           int64
weight        float64
dtype: object

In [19]:
df["fruit"].values.categories

Index(['apple', 'orange'], dtype='object')

In [20]:
df["fruit"].values.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

> Notice that the dtype is NumPy’s int8, an 8-bit signed integer that can take on values from -127 to 128. (Only a single byte is needed to represent a value in memory. 64-bit signed ints would be overkill in terms of memory usage.) Our rough-hewn example resulted in int64 data by default, whereas Pandas is smart enough to downcast categorical data to the smallest numerical dtype possible.

Kategoričen series lahko definiramo že pri inicializaciji

In [23]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

## Better performance with categoricals
Pretvorba se splača če je recimo več kot 50% vrednosti unikatnih...
Če recimo zgolj 30% ali manj unikatnih vrednostih se ne splača pretvoriti

In [22]:
N = 10_000_000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [23]:
labels.head()

0    foo
1    bar
2    baz
3    qux
4    foo
dtype: object

In [24]:
categories =  labels.astype("category")

In [25]:
categories.head()

0    foo
1    bar
2    baz
3    qux
4    foo
dtype: category
Categories (4, object): ['bar', 'baz', 'foo', 'qux']

In [28]:
print(f"{labels.memory_usage(deep=True)/1024/1024} MB")

572.2047119140625 MB


In [29]:
print(f"{categories.memory_usage(deep=True)/1024/1024} MB")

9.53725814819336 MB


## Categorical Methods

In [36]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [44]:
colors = pd.Series(['periwinkle', 'mint green', 'burnt orange',
                     'periwinkle', 'burnt orange', 'rose', 
                     'rose', 'mint green', 'rose', 'navy'])

ccolors = colors.astype('category')

In [ ]:
try:
     ccolors.iloc[5] = 'a new color'
except ValueError as e:
    print(e)

## Example: Using The Pandas Category Data Type

### Data Preparation

First, set up imports and read in all the data:

In [56]:
df_raw = pd.read_csv('data/category_example_data.csv')

In [58]:
df_raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 176 entries, Change_Type to Context_of_Research
dtypes: float64(43), int64(3), object(130)
memory usage: 566.0 MB


> [pandas.DataFrame.from_records](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_records.html)

In [66]:
unique_counts = pd.DataFrame.from_records([(col, df[col].nunique()) for col in df.columns],
                                         columns=['Column_Name', 'Num_Unique'])

In [72]:
cols_to_exclude = ['Program_Year', 'Payment_Publication_Date', 'Date_of_Payment']

In [73]:
for col in df.columns:
    if df[col].nunique() < 600 and col not in cols_to_exclude:
        df[col] = df[col].astype('category')

In [79]:
df.memory_usage(deep=True).sum() / (1024*1024)

51.83827590942383

### Performance

Perform the analysis on the original input dataframe.

In [81]:
%%timeit
df_raw.groupby('Covered_Recipient_Type')['Total_Amount_of_Payment_USDollars'].sum().to_frame()

9.91 ms ± 439 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now, on the dataframe with categorical data:

In [82]:
%%timeit
df.groupby('Covered_Recipient_Type')['Total_Amount_of_Payment_USDollars'].sum().to_frame()

1.91 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Watch Outs

> The real problem is that programmers have spent far too much time worrying about efficiency in the wrong places and at the wrong times; premature optimization is the root of all evil (or at least most of it) in programming.

### General Guidelines


1. Do not assume you need to convert all categorical data to the pandas category data type.
2. If the data set starts to approach an appreciable percentage of your useable memory, then consider using categorical data types.
3. If you have very significant performance concerns with operations that are executed frequently, look at using categorical data.
4. If you are using categorical data, add some checks to make sure the data is clean and complete before converting to the pandas category type. Additionally, check for NaN values after combining or converting dataframes.
